In [1]:
print("all ok")

all ok


In [2]:
from langchain_core.documents import Document

In [3]:
docs=[
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="A hacker discovers reality is a simulation and leads a rebellion against the machines controlling it.",
        metadata={"year": 1999, "director": "Lana Wachowski, Lilly Wachowski", "rating": 8.7, "genre": "science fiction"},
    ),
    Document(
        page_content="A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.",
        metadata={"year": 1994, "rating": 8.5, "genre": "animated"},
    ),
    Document(
        page_content="Batman faces off against the Joker, a criminal mastermind who plunges Gotham into chaos.",
        metadata={"year": 2008, "director": "Christopher Nolan", "rating": 9.0, "genre": "action"},
    ),
    Document(
        page_content="A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.",
        metadata={"year": 2014, "director": "Christopher Nolan", "rating": 8.6, "genre": "science fiction"},
    )
]

In [4]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [5]:
embeddings=OpenAIEmbeddings(model="text-embedding-ada-002")

In [6]:
vectorstore=Chroma(collection_name="movie_data",
                   embedding_function=embeddings)

In [7]:
vectorstore.add_documents(docs)

['61374c17-557e-44be-b00e-5912336acbda',
 '7f7192ed-7c98-43c7-8754-e408f28b2179',
 '3060c846-d393-49b0-9bae-c3ee1c744fd4',
 '031a6b9a-6e07-40d8-ab9f-718bca1b8a70',
 'dad4934c-6588-4610-a662-e4e657f6d111',
 'c052662b-04b0-4ce5-a58c-ea5de5f50c69',
 'afdfa297-bb9c-409a-a24b-cb5687a60200',
 '0953bdcf-4f92-45d3-aaea-344aa0b53d63',
 '4ec84d6d-bca1-4628-b51d-b65464b06f83']

In [8]:
retriever=vectorstore.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [9]:
question1 = "Which 1994 animated movie has a rating of 8.5?"
question2 = "Which movie features Batman facing off against the Joker and who directed it?"
question3 = "What genre is the movie 'The Matrix' and who directed it?"

In [10]:
retriever.invoke(question1)

[Document(id='dad4934c-6588-4610-a662-e4e657f6d111', metadata={'year': 1995, 'genre': 'animated'}, page_content='Toys come alive and have a blast doing so'),
 Document(id='afdfa297-bb9c-409a-a24b-cb5687a60200', metadata={'year': 1994, 'genre': 'animated', 'rating': 8.5}, page_content='A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.'),
 Document(id='61374c17-557e-44be-b00e-5912336acbda', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose')]

In [11]:
retriever.invoke(question2)

[Document(id='0953bdcf-4f92-45d3-aaea-344aa0b53d63', metadata={'year': 2008, 'genre': 'action', 'rating': 9.0, 'director': 'Christopher Nolan'}, page_content='Batman faces off against the Joker, a criminal mastermind who plunges Gotham into chaos.'),
 Document(id='3060c846-d393-49b0-9bae-c3ee1c744fd4', metadata={'director': 'Satoshi Kon', 'year': 2006, 'rating': 8.6}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(id='7f7192ed-7c98-43c7-8754-e408f28b2179', metadata={'year': 2010, 'rating': 8.2, 'director': 'Christopher Nolan'}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...')]

In [12]:
retriever.invoke(question3)

[Document(id='c052662b-04b0-4ce5-a58c-ea5de5f50c69', metadata={'director': 'Lana Wachowski, Lilly Wachowski', 'year': 1999, 'rating': 8.7, 'genre': 'science fiction'}, page_content='A hacker discovers reality is a simulation and leads a rebellion against the machines controlling it.'),
 Document(id='3060c846-d393-49b0-9bae-c3ee1c744fd4', metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(id='4ec84d6d-bca1-4628-b51d-b65464b06f83', metadata={'rating': 8.6, 'director': 'Christopher Nolan', 'year': 2014, 'genre': 'science fiction'}, page_content="A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.")]

In [13]:
template="""You are a help assistant. Answer the user query based on provided context.
if response is not avaliable for the query then tell you don't know.
Query: {query}
Context: {context}
Answer:
"""

In [14]:
prompt=PromptTemplate.from_template(template=template)

In [15]:
llm=ChatOpenAI(model='gpt-4o')

In [16]:
chain=(
    {"context": retriever,"query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
response=chain.invoke(question1)
response

'The 1994 animated movie with a rating of 8.5 is about a young lion prince who flees his kingdom only to learn the true meaning of responsibility and bravery.'

In [18]:
response=chain.invoke(question2)
response

'The movie featuring Batman facing off against the Joker is directed by Christopher Nolan.'

In [19]:
response=chain.invoke(question3)
response

"The genre of the movie 'The Matrix' is science fiction, and it was directed by Lana Wachowski and Lilly Wachowski."

## Self-Query-Retrieval

In [20]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [21]:

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
   

In [22]:
metadata_field_info

[AttributeInfo(name='genre', description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']", type='string'),
 AttributeInfo(name='year', description='The year the movie was released', type='integer'),
 AttributeInfo(name='director', description='The name of the movie director', type='string'),
 AttributeInfo(name='rating', description='A 1-10 rating for the movie', type='float')]

In [23]:
document_content_description = "Brief summary of a movie"

In [24]:
from langchain.chains.query_constructor.base import StructuredQueryOutputParser,get_query_constructor_prompt

In [25]:
prompt=get_query_constructor_prompt(document_content_description,metadata_field_info)

In [26]:
prompt

FewShotPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\")), lt(\\"length\\", 180), eq(\\"genre\\", \\"pop\\"))"\n}}\n```'}, {'i': 2, 'data_source': '```json\n{{\n    "con

In [27]:
import lark
output_parser=StructuredQueryOutputParser.from_components()

In [30]:
query_constructor= prompt | llm | output_parser


In [31]:
query_constructor.invoke(
    {"query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"})

StructuredQuery(query='taxi drivers', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='year', value=2000), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson')]), limit=None)

In [32]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

In [34]:
retriever=SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [35]:
retriever.invoke(question1)

[Document(id='afdfa297-bb9c-409a-a24b-cb5687a60200', metadata={'rating': 8.5, 'year': 1994, 'genre': 'animated'}, page_content='A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.')]

In [36]:
retriever.invoke("What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated")

[Document(id='dad4934c-6588-4610-a662-e4e657f6d111', metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so'),
 Document(id='afdfa297-bb9c-409a-a24b-cb5687a60200', metadata={'year': 1994, 'genre': 'animated', 'rating': 8.5}, page_content='A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.')]

In [46]:
from operator import itemgetter

In [50]:

template = """Answer the question based only on the following context:
{context}

Question: {query}
"""
prompt_llm=PromptTemplate(template=template)

In [51]:
self_query_chain=(
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt_llm
    | llm
    | StrOutputParser()
    
)

In [52]:

text_reply = self_query_chain.invoke("What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated")
text_reply

'The movie "Toys come alive and have a blast doing so" fits your criteria. It is an animated movie released in 1995.'

In [53]:
text_reply = self_query_chain.invoke("Tell me about the movie which have rating more than 7.")
text_reply

"The documents describe several movies with a rating higher than 7:\n\n1. **Document ID: 0953bdcf-4f92-45d3-aaea-344aa0b53d63**\n   - **Title**: Not provided\n   - **Director**: Christopher Nolan\n   - **Year**: 2008\n   - **Rating**: 9.0\n   - **Genre**: Action\n   - **Plot**: Batman faces off against the Joker, a criminal mastermind who plunges Gotham into chaos.\n\n2. **Document ID: afdfa297-bb9c-409a-a24b-cb5687a60200**\n   - **Title**: Not provided\n   - **Year**: 1994\n   - **Rating**: 8.5\n   - **Genre**: Animated\n   - **Plot**: A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.\n\n3. **Document ID: 4ec84d6d-bca1-4628-b51d-b65464b06f83**\n   - **Title**: Not provided\n   - **Director**: Christopher Nolan\n   - **Year**: 2014\n   - **Rating**: 8.6\n   - **Genre**: Science Fiction\n   - **Plot**: A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.\n\n4. **Document ID: 61374c17-557e-44

In [54]:
print(text_reply)

The documents describe several movies with a rating higher than 7:

1. **Document ID: 0953bdcf-4f92-45d3-aaea-344aa0b53d63**
   - **Title**: Not provided
   - **Director**: Christopher Nolan
   - **Year**: 2008
   - **Rating**: 9.0
   - **Genre**: Action
   - **Plot**: Batman faces off against the Joker, a criminal mastermind who plunges Gotham into chaos.

2. **Document ID: afdfa297-bb9c-409a-a24b-cb5687a60200**
   - **Title**: Not provided
   - **Year**: 1994
   - **Rating**: 8.5
   - **Genre**: Animated
   - **Plot**: A young lion prince flees his kingdom only to learn the true meaning of responsibility and bravery.

3. **Document ID: 4ec84d6d-bca1-4628-b51d-b65464b06f83**
   - **Title**: Not provided
   - **Director**: Christopher Nolan
   - **Year**: 2014
   - **Rating**: 8.6
   - **Genre**: Science Fiction
   - **Plot**: A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.

4. **Document ID: 61374c17-557e-44be-b00e-5912336acbda**
   